从Bert开始，NLP任务都改用subword的分词方法，主要包括：BPE、WordPiece、ULM

# BPE训练

训练流程

1. 将数据集拆成词，并统计词频
2. 首先将数据集拆成词，并统计词频 （为了区分词的开始、结束，可以末尾添加</w>，或者在句中添加##等特殊符号）
  举例如下：
      {"l o w </w>":5, "l o w e r </w>":2, "n e w e s t </w>": 6, "w i d e s t </w>": 3}
      词典大小：{"l","o","w","e","r","n","s","t","i","d","</w>"}
3. 从统计出的词频中，统计bigram最多的次数，组成新的单元
lo: 5+2=7, ow: 5+2=7, w</w>: 5
we: 2+6=8, er: 2, r</w>: 2
ne: 6, ew: 6, es: 6+3=9, st: 6+3=9, t</w>: 6+3=9
wi: 3, id: 3, de: 3
es、st、t</w>都为9，可任选一个，变成{"l o w </w>":5, "l o w e r </w>":2, "n e w es t </w>": 6, "w i d es t </w>": 3}
词典大小：{"l","o","w","e","r","n","es","t","i","d", "</w>"}
4. 继续第3步，直到：达到预设的迭代次数 或 达到预设的Subword词表大小 或 下一个最高频的字节对出现频率为1

In [ ]:
# 训练代码实现如下：

# 首先我们需要一个简单的分词器，将句子拆分成单词（如根据空格、标点进行拆分）
import toolz, re
from collections import Counter


def wordpunct_tokenize(text):
    _pattern = r"\w+|[^\w\s]+"
    _regexp = re.compile(_pattern, flags=re.UNICODE | re.MULTILINE | re.DOTALL)
    return _regexp.findall(text)


In [ ]:
# 词典训练过程如下（BPETokenizer.fit()）

class BPETokenizer():
    special = ['<UNK>', '<PAD>', '<END>', '<MASK>']

    def __init__(self, vocab_size=1000, lowercase=True, basic_tokenizer=wordpunct_tokenize):
        self.lowercase = lowercase
        self.vocab_size = vocab_size
        self.basic_tokenizer = basic_tokenizer

    def fit(self, corpus: list, max_steps=10000, out_fn='vocab.txt'):
        '''
        分词器训练，返回训练得到的vocabulary
        '''

        ############### 统计初始词典 ###############
        if self.lowercase:
            corpus = [s.lower() for s in corpus]
        word_corpus = Counter([tuple(data) + ("</w>",) for data in toolz.concat(map(self.basic_tokenizer, corpus))])
        vocab = self._count_vocab(word_corpus)

        ############### 逐步合并初始词典中的高频二元组 ###############
        for i in range(max_steps):
            word_corpus, bi_cnt = self._fit_step(word_corpus)
            vocab = self._count_vocab(word_corpus)
            if len(vocab) >= self.vocab_size or bi_cnt < 0: break

        ############### 将一些特殊词加入最终的词典 ###############
        for s in self.special:
            if s not in vocab:
                vocab.insert(0, (s, 99999))

        ############### 导出词典 ###############
        with open(out_fn, 'w') as f:
            f.write('\n'.join([w for w, _ in vocab]))
		self.vocab = [token for token, _ in vocab]
        return vocab

    def _count_vocab(self, word_corpus):
        _r = Counter([data for data in toolz.concat([word * cnt for word, cnt in word_corpus.items()])])
        _r = sorted(_r.items(), key=lambda x: -x[1])
        return _r

    def _fit_step(self, word_corpus):
        ngram = 2
        bigram_counter = Counter()

        ############### 以步长1，窗口尺寸2，在每个单词上滚动，统计二元组频次 ###############
        for token, count in word_corpus.items():
            if len(token) < 2: continue
            for bigram in toolz.sliding_window(ngram, token):
                bigram_counter[bigram] += count

        ############### 选出频次最大的二元组 ###############
        if len(bigram_counter) > 0:
            max_bigram = max(bigram_counter, key=bigram_counter.get)
        else:
            return word_corpus, -1
        bi_cnt = bigram_counter.get(max_bigram)

        ############### 从corpus中将最大二元组出现的地方替换成一个token ###############
        for token in word_corpus:
            _new_token = tuple(' '.join(token).replace(' '.join(max_bigram), ''.join(max_bigram)).split(' '))
            if _new_token != token:
                word_corpus[_new_token] = word_corpus[token]
                word_corpus.pop(token)
        return word_corpus, bi_cnt

- 统计初始词典
  - 首先如果设置了lowercase，会将大/小写同等对待（统一转换成小写）
  - 然后通过对句子简单分词，并数量统计，得到Counter结构，如Counter({('n', 'e', 'w', 'e', 's', 't', '</w>'): 6, ('l', 'o', 'w', '</w>'): 5, ('w', 'i', 'd', 'e', 's', 't', '</w>'): 3, ('l', 'o', 'w', 'e', 'r', '</w>'): 2})
  - 在_count_vocab中统计处目前的词典为[('e', 17), ('w', 16), ('</w>', 16), ('s', 9), ('t', 9), ('l', 7), ('o', 7), ('n', 6), ('i', 3), ('d', 3), ('r', 2)]
- 持续迭代合并词典中的高频二元组（过程见_fit_step()），并更新vocab。直到 超过迭代步数 或 词典大小满足要求 或 已经没有可合并元素
  - 在corpus的每个word中，以步长1，窗口尺寸2，统计出所有二元组token的频次
  - 将最大二元组出现的地方合并成一个token
- 最后是添加一些特殊词并导出词典


# BPE分词
分词是利用上一步训练好的vocab，将句子切分成词典中的token，分词是一个匹配最长子串的过程

- 首先还是利用简单分词器self.basic_tokenzier，将句子分成单词序列
- 然后对每个单词，从后往前，依次找到包含在vocab中的最长sub_token
  - 对于某个单词，如果任何sub_token都不包含在vocab中，那么当做未登录词"<UNK>"


分词代码如下：

- 重点关注tokenize、encode、decode


In [ ]:
def tokenize(self, text: str, add_post='</w>'):
    '''
    将text转换成tokens
    '''

    all_tokens = []
    if self.lowercase: text = text.lower()
    new_token = []

    ############### 简单分词，并遍历token ###############
    for token in self.basic_tokenizer(text):
        token = list(token)
        if add_post:
            token = token + [add_post]
        start, end = 0, len(token)

        ############### 查找最长sub_token ###############
        while start < end:
            sub_token = ''.join(token[start:end])
            if sub_token in self.vocab:
                new_token.append(sub_token)
                start = end
                end = len(token)
            elif end - start == 1:
                new_token.append('<UNK>')
                start = end
                end = len(token)
            else:
                end -= 1
    all_tokens.extend(new_token)
    return all_tokens

def encode(self, text: str):
    '''
    将text转换成token_ids
    '''
    tokens_list = self.tokenize(text)
    ids_list = [list(map(lambda x: self._token2id[x], tokens)) for tokens in tokens_list]
    return ids_list

def decode(self, token_ids):
    '''
    将token_ids还原成text
    '''
    sentences = []
    for ids in token_ids:
        sentence = list(map(lambda x: self._id2token[x], ids))
        sentence = ''.join(sentence).replace('</w>', ' ')
        sentences.append(sentence)
    return sentences

def _token2id(self, token):
    if token in self.vocab:
        return self.vocab.index(token)
    return self.vocab.index('<UNK>')

def _id2token(self, id):
    return self.vocab[id]